# DYNAMIC PARKING PRICING SYSTEM - Final Capstone Project

In [30]:
!pip install pathway bokeh --quiet
import warnings
warnings.filterwarnings('ignore')

In [32]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource
import panel as pn

output_notebook()
pn.extension()

print("🚀 Dynamic Parking Pricing System Starting...")

🚀 Dynamic Parking Pricing System Starting...


### CORE PRICING ENGINE

In [33]:

class DynamicParkingPricer:
    def __init__(self, base_price=10.0):
        self.base_price = base_price
        self.vehicle_multipliers = {'car': 1.0, 'truck': 1.8, 'bike': 0.5, 'cycle': 0.3}
        self.traffic_multipliers = {'low': 0.8, 'average': 1.0, 'high': 1.3}

    def model_1_linear_pricing(self, row, previous_price=None):
        if previous_price is None:
            previous_price = self.base_price
        alpha = 0.1
        occupancy_rate = row['Occupancy'] / row['Capacity']
        new_price = previous_price + alpha * occupancy_rate
        return max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

    def model_2_demand_based_pricing(self, row):
        occupancy_factor = 0.4 * (row['Occupancy'] / row['Capacity'])
        queue_factor = 0.1 * min(row['QueueLength'] / 10.0, 1.0)
        traffic_factor = 0.15 * self.traffic_multipliers.get(row['TrafficConditionNearby'], 1.0)
        special_day_factor = 0.4 * row['IsSpecialDay']
        vehicle_factor = 0.1 * self.vehicle_multipliers.get(row['VehicleType'], 1.0)

        demand = occupancy_factor + queue_factor + traffic_factor + special_day_factor + vehicle_factor
        normalized_demand = min(max(demand, 0), 2.0)
        price = self.base_price * (1 + 0.5 * normalized_demand)
        return max(self.base_price * 0.5, min(price, self.base_price * 2.0))

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        R = 6371
        lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
        dlat, dlon = lat2 - lat1, lon2 - lon1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        return R * 2 * math.asin(math.sqrt(a))

    def model_3_competitive_pricing(self, row, all_current_data):
        base_price = self.model_2_demand_based_pricing(row)
        current_lat, current_lon = row['Latitude'], row['Longitude']
        nearby_prices = []

        for _, other_row in all_current_data.iterrows():
            if other_row['SystemCodeNumber'] == row['SystemCodeNumber']:
                continue
            distance = self.haversine_distance(current_lat, current_lon,
                                             other_row['Latitude'], other_row['Longitude'])
            if distance <= 2.0:
                other_price = self.model_2_demand_based_pricing(other_row)
                nearby_prices.append(other_price)

        if nearby_prices:
            avg_competitor_price = np.mean(nearby_prices)
            occupancy_rate = row['Occupancy'] / row['Capacity']
            if occupancy_rate > 0.8 and base_price > avg_competitor_price * 1.1:
                competitive_price = avg_competitor_price * 1.05
            elif base_price < min(nearby_prices) * 0.9:
                competitive_price = min(nearby_prices) * 0.95
            else:
                competitive_price = base_price
            return max(self.base_price * 0.5, min(competitive_price, self.base_price * 2.0))
        return base_price

# ============================================================================
# DATA PROCESSING
# ============================================================================

def process_parking_data(csv_file="dataset.csv"):
    print("📊 Loading data...")
    df = pd.read_csv("/content/drive/MyDrive/SA Capstone Project/dataset.csv")
    df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                    format='%d-%m-%Y %H:%M:%S')
    df = df.sort_values('Timestamp').reset_index(drop=True)
    df['Hour'] = df['Timestamp'].dt.hour
    df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

    print(f"✅ Loaded {len(df):,} records from {df['SystemCodeNumber'].nunique()} locations")

    pricer = DynamicParkingPricer()
    results = []

    print("⚡ Running pricing simulation...")
    for location in df['SystemCodeNumber'].unique():
        location_data = df[df['SystemCodeNumber'] == location].copy()
        previous_price = pricer.base_price

        for idx, row in location_data.iterrows():
            linear_price = pricer.model_1_linear_pricing(row, previous_price)
            demand_price = pricer.model_2_demand_based_pricing(row)
            current_time_data = df[df['Timestamp'] == row['Timestamp']]
            competitive_price = pricer.model_3_competitive_pricing(row, current_time_data)

            results.append({
                'timestamp': row['Timestamp'],
                'location': row['SystemCodeNumber'],
                'occupancy': row['Occupancy'],
                'capacity': row['Capacity'],
                'occupancy_rate': row['OccupancyRate'],
                'vehicle_type': row['VehicleType'],
                'traffic': row['TrafficConditionNearby'],
                'queue_length': row['QueueLength'],
                'is_special_day': row['IsSpecialDay'],
                'linear_price': round(linear_price, 2),
                'demand_price': round(demand_price, 2),
                'competitive_price': round(competitive_price, 2),
                'final_price': round(demand_price, 2)
            })
            previous_price = demand_price

    results_df = pd.DataFrame(results)
    print(f"✅ Generated {len(results_df):,} pricing decisions")
    return results_df

# ============================================================================
# ANALYTICS AND REPORTING
# ============================================================================

def generate_comprehensive_report(results_df):
    print("📈 Generating analytics...")
    total_records = len(results_df)
    num_locations = results_df['location'].nunique()
    avg_price = results_df['final_price'].mean()
    avg_occupancy = results_df['occupancy_rate'].mean() * 100

    static_revenue = total_records * 10
    dynamic_revenue = results_df['final_price'].sum()
    revenue_increase = ((dynamic_revenue - static_revenue) / static_revenue) * 100

    results_df['hour'] = pd.to_datetime(results_df['timestamp']).dt.hour
    hourly_stats = results_df.groupby('hour')['final_price'].mean()
    peak_hour = hourly_stats.idxmax()
    peak_price = hourly_stats.max()

    results_df['occ_bin'] = pd.cut(results_df['occupancy_rate'],
                                  bins=[0, 0.3, 0.6, 0.9, 1.0],
                                  labels=['Low (0-30%)', 'Medium (30-60%)', 'High (60-90%)', 'Full (90%+)'])
    occupancy_pricing = results_df.groupby('occ_bin')['final_price'].mean()

    vehicle_pricing = results_df.groupby('vehicle_type')['final_price'].mean().sort_values(ascending=False)
    traffic_pricing = results_df.groupby('traffic')['final_price'].mean().sort_values(ascending=False)

    model_comparison = {
        'Linear': results_df['linear_price'].mean(),
        'Demand-Based': results_df['demand_price'].mean(),
        'Competitive': results_df['competitive_price'].mean()
    }

    print("\n" + "="*80)
    print("DYNAMIC PRICING SYSTEM - COMPREHENSIVE ANALYTICS REPORT")
    print("="*80)
    print(f"\n📊 OVERALL PERFORMANCE:")
    print(f"• Records Processed: {total_records:,}")
    print(f"• Parking Locations: {num_locations}")
    print(f"• Average Price: ${avg_price:.2f}")
    print(f"• Average Occupancy: {avg_occupancy:.1f}%")

    print(f"\n💰 REVENUE IMPACT:")
    print(f"• Static Revenue ($10/slot): ${static_revenue:,.2f}")
    print(f"• Dynamic Revenue: ${dynamic_revenue:,.2f}")
    print(f"• Revenue Increase: {revenue_increase:+.1f}%")
    print(f"• Additional Revenue: ${dynamic_revenue - static_revenue:,.2f}")

    print(f"\n⏰ PEAK PRICING:")
    print(f"• Peak Hour: {peak_hour}:00")
    print(f"• Peak Price: ${peak_price:.2f}")

    print(f"\n🚗 OCCUPANCY-BASED PRICING:")
    for level, price in occupancy_pricing.items():
        print(f"• {level}: ${price:.2f}")

    print(f"\n🚙 VEHICLE TYPE PRICING:")
    for vehicle, price in vehicle_pricing.items():
        print(f"• {vehicle.capitalize()}: ${price:.2f}")

    print(f"\n🚦 TRAFFIC IMPACT:")
    for traffic, price in traffic_pricing.items():
        print(f"• {traffic.capitalize()}: ${price:.2f}")

    print(f"\n🔄 MODEL COMPARISON:")
    for model, price in model_comparison.items():
        print(f"• {model}: ${price:.2f}")

    print("\n" + "="*80)

    return {
        'revenue_increase': revenue_increase,
        'peak_hour': peak_hour,
        'model_comparison': model_comparison
    }

# ============================================================================
# BOKEH VISUALIZATION DASHBOARD
# ============================================================================

def create_bokeh_dashboard(results_df):
    hourly_avg = results_df.groupby(results_df['timestamp'].dt.hour)['final_price'].mean().reset_index()
    source_hourly = ColumnDataSource(hourly_avg)
    source = ColumnDataSource(results_df)

    # 1. Hourly Pricing Trends
    p1 = figure(title="Hourly Pricing Patterns", x_axis_label='Hour of Day', y_axis_label='Average Price ($)',
                width=600, height=300, sizing_mode='scale_width')
    p1.line('timestamp', 'final_price', source=source_hourly, line_width=2, color='navy')
    p1.circle('timestamp', 'final_price', source=source_hourly, size=8, color='red')
    p1.xaxis.ticker = list(range(24))

    # 2. Occupancy Rate vs Price Scatter Plot
    p2 = figure(title="Occupancy Rate vs Price", x_axis_label='Occupancy Rate (%)', y_axis_label='Price ($)',
                width=600, height=300, tools='pan,wheel_zoom,box_zoom,reset', sizing_mode='scale_width')
    p2.circle(results_df['occupancy_rate']*100, results_df['final_price'], size=5, alpha=0.5, color='green')

    # 3. Pricing by Vehicle Type Bar Chart
    vehicle_avg = results_df.groupby('vehicle_type')['final_price'].mean().reset_index()
    source_vehicle = ColumnDataSource(vehicle_avg)
    p3 = figure(x_range=vehicle_avg['vehicle_type'], title="Pricing by Vehicle Type",
                y_axis_label='Average Price ($)', width=600, height=300, sizing_mode='scale_width')
    p3.vbar(x='vehicle_type', top='final_price', width=0.6, source=source_vehicle, color='orange')

    # 4. Pricing by Traffic Condition Bar Chart
    traffic_avg = results_df.groupby('traffic')['final_price'].mean().reset_index()
    source_traffic = ColumnDataSource(traffic_avg)
    p4 = figure(x_range=traffic_avg['traffic'], title="Pricing by Traffic Condition",
                y_axis_label='Average Price ($)', width=600, height=300, sizing_mode='scale_width')
    p4.vbar(x='traffic', top='final_price', width=0.6, source=source_traffic, color='purple')

    layout = column(p1, p2, row(p3, p4), sizing_mode='scale_width')
    return layout

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def run_complete_dynamic_pricing_system():
    print("🚀 DYNAMIC PRICING SYSTEM FOR URBAN PARKING LOTS")
    print("="*70)

    try:
        results_df = process_parking_data()
        analytics = generate_comprehensive_report(results_df)
        results_df.to_csv('dynamic_pricing_results.csv', index=False)
        print("\n💾 Results saved to 'dynamic_pricing_results.csv'")
        print("\n🎉 DYNAMIC PRICING SYSTEM EXECUTION COMPLETE!")
        print("✅ Revenue increase achieved:", f"{analytics['revenue_increase']:.1f}%")
        print("✅ Peak pricing hour:", f"{analytics['peak_hour']}:00")
        print("="*70)
        return results_df, analytics
    except FileNotFoundError:
        print("❌ Error: dataset.csv not found!")
        print("Please ensure the dataset.csv file is in the same directory.")
        return None, None
    except Exception as e:
        print(f"❌ Error occurred: {e}")
        return None, None

# ============================================================================
# EXECUTE THE SYSTEM
# ============================================================================

results, analytics = run_complete_dynamic_pricing_system()

if results is not None:
    print("\n🎯 System ready! Results available in 'results' variable.")
    print("📊 Analytics available in 'analytics' variable.")
    # Show Bokeh dashboard
    dashboard = create_bokeh_dashboard(results)
    pn.panel(dashboard).servable()
else:
    print("\n⚠️ System execution failed. Please check the error messages above.")

🚀 DYNAMIC PRICING SYSTEM FOR URBAN PARKING LOTS
📊 Loading data...
✅ Loaded 18,368 records from 14 locations
⚡ Running pricing simulation...
✅ Generated 18,368 pricing decisions
📈 Generating analytics...

DYNAMIC PRICING SYSTEM - COMPREHENSIVE ANALYTICS REPORT

📊 OVERALL PERFORMANCE:
• Records Processed: 18,368
• Parking Locations: 14
• Average Price: $12.74
• Average Occupancy: 50.9%

💰 REVENUE IMPACT:
• Static Revenue ($10/slot): $183,680.00
• Dynamic Revenue: $234,029.70
• Revenue Increase: +27.4%
• Additional Revenue: $50,349.70

⏰ PEAK PRICING:
• Peak Hour: 13:00
• Peak Price: $13.29

🚗 OCCUPANCY-BASED PRICING:
• Low (0-30%): $12.32
• Medium (30-60%): $12.59
• High (60-90%): $13.07
• Full (90%+): $13.52

🚙 VEHICLE TYPE PRICING:
• Truck: $13.20
• Car: $12.78
• Bike: $12.54
• Cycle: $12.42

🚦 TRAFFIC IMPACT:
• High: $13.31
• Average: $12.85
• Low: $12.35

🔄 MODEL COMPARISON:
• Linear: $12.79
• Demand-Based: $12.74
• Competitive: $12.74


💾 Results saved to 'dynamic_pricing_results.cs

In [35]:
def create_bokeh_dashboard(results_df):
    # Prepare data sources
    hourly_avg = results_df.groupby(results_df['timestamp'].dt.hour)['final_price'].mean().reset_index()
    source_hourly = ColumnDataSource(hourly_avg)

    source = ColumnDataSource(results_df)

    # 1. Hourly Pricing Trends
    p1 = figure(title="Hourly Pricing Patterns", x_axis_label='Hour of Day', y_axis_label='Average Price ($)',
                width=600, height=300, sizing_mode='scale_width')
    p1.line('timestamp', 'final_price', source=source_hourly, line_width=2, color='navy')
    p1.circle('timestamp', 'final_price', source=source_hourly, size=8, color='red')
    p1.xaxis.ticker = list(range(24))

    # 2. Occupancy Rate vs Price Scatter Plot
    p2 = figure(title="Occupancy Rate vs Price", x_axis_label='Occupancy Rate (%)', y_axis_label='Price ($)',
                width=600, height=300, tools='pan,wheel_zoom,box_zoom,reset', sizing_mode='scale_width')
    p2.circle(results_df['occupancy_rate']*100, results_df['final_price'], size=5, alpha=0.5, color='green')

    # 3. Pricing by Vehicle Type Bar Chart
    vehicle_avg = results_df.groupby('vehicle_type')['final_price'].mean().reset_index()
    source_vehicle = ColumnDataSource(vehicle_avg)
    p3 = figure(x_range=vehicle_avg['vehicle_type'], title="Pricing by Vehicle Type",
                y_axis_label='Average Price ($)', width=600, height=300, sizing_mode='scale_width')
    p3.vbar(x='vehicle_type', top='final_price', width=0.6, source=source_vehicle, color='orange')

    # 4. Pricing by Traffic Condition Bar Chart
    traffic_avg = results_df.groupby('traffic')['final_price'].mean().reset_index()
    source_traffic = ColumnDataSource(traffic_avg)
    p4 = figure(x_range=traffic_avg['traffic'], title="Pricing by Traffic Condition",
                y_axis_label='Average Price ($)', width=600, height=300, sizing_mode='scale_width')
    p4.vbar(x='traffic', top='final_price', width=0.6, source=source_traffic, color='purple')

    # Arrange plots in a layout
    layout = column(p1, p2, row(p3, p4), sizing_mode='scale_width')
    return layout

In [36]:
dashboard = create_bokeh_dashboard(results_df)
pn.panel(dashboard).servable()


Bokeh(Column)

In [38]:
output_notebook()

p = figure(title="Simple Line Example", x_axis_label='x', y_axis_label='y', sizing_mode='scale_width')
p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)
show(p)
